## Importando bibliotecas 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from scipy.stats import moment

# Statsmodels
import statsmodels

from sklearn.metrics import mean_squared_error
import plotly.express as px

import time
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler

import os
import random

import pmdarima as pm

import pycountry_convert as pc

## Pre-processamento do dataset

### Importando dataset
Vamos importar o dataset e realizar o pre processamento de dados numéricos e dados faltantes

In [2]:
dataset_path = 'dataset.xlsx'
xl_file = pd.ExcelFile(dataset_path)
xl_file.sheet_names

['ReadMe', 'ExpNotes', 'Data Issues', 'Data', 'Totals']

In [3]:
sheet_name = 'Data'
dataset = pd.read_excel(dataset_path, sheet_name=sheet_name)

In [4]:
# Pegando a lista de colunas com valores numericos a serem tratadas
numeric_cols = list(dataset.columns[3:])

# Formatando valores numericos
dataset[numeric_cols] = dataset[numeric_cols].apply(lambda x:  pd.to_numeric(x, errors='coerce'))

## Tratamento de valores faltantes em séries temporais
Existem várias formas de tratar de valores faltantes, desde descartar as linhas com valores faltantes, substituir pela média, mediana e até utilizar de interpolação para tratar esses valores. Em séries temporais somente o ultimo seria de alguma valia para manter o comportamento da série temporal e sua distribuição. Outra forma para tratar valores faltantes, especificamente para séries temporais, seria substituir pelo ultimo ou pelo proximo valor. Nesse case foi escolido a substituição pelo ultimo valor como forma de tratar outlier.

Dessa forma, Para tratar os valores faltantes vamos utilizar o `bfill` do pandas para utilizar o ultimo valor conhecido para substituir os valores `NaN`.

Como cada país tem um comportamento de série temporal diferente é necessário tratar os valores faltantes por país, para não acabar inserindo dados relativos de uma distribuição diferente no valor faltante.

In [5]:
unique_countries = list(dataset['Country to/from'].unique())

for country in unique_countries:
   dataset.loc[dataset['Country to/from'] == country] =  dataset.loc[dataset['Country to/from'] == country].bfill()

In [6]:
for col in list(dataset.columns):
    print(f'Coluna {col} | Quantidade de NaN {dataset[col].isnull().sum()} ')


Coluna Month | Quantidade de NaN 0 
Coluna Scheduled Operator | Quantidade de NaN 0 
Coluna Country to/from | Quantidade de NaN 0 
Coluna Passengers In | Quantidade de NaN 135 
Coluna Freight In | Quantidade de NaN 33 
Coluna Mail In | Quantidade de NaN 33 
Coluna Passengers Out | Quantidade de NaN 88 
Coluna Freight Out | Quantidade de NaN 32 
Coluna Mail Out | Quantidade de NaN 32 
Coluna Year | Quantidade de NaN 0 


In [7]:
# Formatando o campo de data
dataset['date'] = pd.to_datetime(dataset['Month'], format="%Y-%m-%d")
dataset.head()

,Month,Scheduled Operator,Country to/from,Passengers In,Freight In,Mail In,Passengers Out,Freight Out,Mail Out,Year,date
0,2009-01-01,Aerolineas Argentinas,Argentina,3021.0,4.313,0.600,1959.0,8.311,0.000,2009,2009-01-01
1,2009-01-01,Aerolineas Argentinas,New Zealand,627.0,76.260,0.000,1821.0,68.539,0.000,2009,2009-01-01
2,2009-01-01,Air Caledonie,New Caledonia,6658.0,4.918,0.645,5365.0,68.621,1.291,2009,2009-01-01
3,2009-01-01,Air Canada,Canada,7489.0,174.828,0.004,6424.0,105.191,0.016,2009,2009-01-01
4,2009-01-01,Air China,China,12458.0,201.314,18.569,11163.0,142.408,2.930,2009,2009-01-01


## Inserindo a noção de continente de partida no dataset
Para isso será utilizado pycountry_convert, porém como vamos ver mais adiante isso pode nos causar alguns erros, como países os quais não conseguem ser identificados pelo módulo. Assim foi necessário o mapeamento manual dos países os quais não tiveram continentes correspondentes.

In [8]:
# inserindo coluna de continente de partida
def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    except (KeyError, TypeError):
        return 'error'
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

In [9]:
dataset['Continent'] = dataset['Country to/from'].apply(lambda x: country_to_continent(x))

In [10]:
# Avaliando causas de erro na transformação de país de origem para continente de origem
country_error_samples = list(dataset.loc[dataset['Continent'] == 'error']['Country to/from'].unique())
country_error_samples

['Tahiti', 'Korea', 'UK', 'Hong Kong (SAR)', 'Western Samoa', 'Reunion']

In [11]:
# Criando mapa manual das causas de erro
country_error_map = {'Tahiti'         : 'Oceania',
                     'Korea'          : 'Asia',
                     'UK'             : 'Europe',
                     'Hong Kong (SAR)': 'Asia',
                     'Western Samoa'  : 'Oceania',
                     'Reunion'        : 'Africa'
}

# Aplicando o mapa no dataset
for country in list(country_error_map.keys()):
    dataset.loc[dataset['Country to/from'] == country, 'Continent'] = country_error_map[country]

# Análise exploratória

## Inserindo a noção de quartil no dataframe
Vamos agora inserir a noção de quartil no dataframe e verificar a média para cada quartil, para verificar se existe uma diferença na média do número de passageiros de acordo com o quartil

In [12]:
dataset['year']    = dataset['date'].dt.year
dataset['Month']   = dataset['date'].dt.month
dataset['day']     = dataset['date'].dt.dayofyear
dataset['quarter'] = dataset['date'].dt.quarter

## Verificando a relação de quartil e número de passageiros

In [13]:
def plot_mean_by_col(col):
    bar_df = {"mean": []}
    bar_df[col] = []

    for value in range(dataset[col].min(),dataset[col].max() + 1):
        bar_df[col].append(value)
        bar_df["mean"].append(dataset[(dataset[col] == value)]['Passengers In'].mean())

    fig = px.bar(bar_df, x=col, y='mean', color=col,
                labels={'Mean value per quarter'}, height=400)
    fig.show()

plot_mean_by_col('quarter')

## Correlação de ano em relação ao número de passageiros

In [14]:
plot_mean_by_col('year')

Analisando-se o gráfico acima é possível notar a influencia clara da pandemia do `covid-19` em relação ao número de passageiros. Até o ano de 2019 era possível notar uma tendência de subida na média de passageiros, porém em 2020 ja foi possível notar a queda considerável do número de passageiros.

## Analisando a série temporal de um País
Vamos agora selecionar uma série temporal e realizar a inspeção visual.

In [38]:
item_train_set = dataset[(dataset['Country to/from'] == 'China')]

In [39]:
item_train_set.head()

,date,Passengers In,Freight In,Mail In,Passengers Out,Freight Out,Mail Out
0,2009-01-01,43656.0,807.783,59.681,34202.0,401.882,17.424
1,2009-02-01,45838.0,1276.580,65.076,35039.0,520.338,16.089
2,2009-03-01,36534.0,2041.685,102.511,30885.0,762.905,20.620
3,2009-04-01,33954.0,2111.625,126.266,32342.0,682.671,18.486
4,2009-05-01,25482.0,1706.501,149.295,25077.0,538.913,21.564


## The data behavior

In [41]:
fig = px.line(item_train_set, x='date', y='Passengers In')
fig.show()

In [ ]:
numeric_cols = ['Passengers In','Freight In','Mail In','Passengers Out','Freight Out','Mail Out']
item_train_set = item_train_set.groupby(item_train_set.date.dt.date)[numeric_cols].sum()
item_train_set = item_train_set.reset_index()

## Data Autocorrelation plot and analysis

By selecting only the first month in the correlation plot we can visualize that our sales series presents a weekly peak, this is a strong indicator that the sales behavior have a weekly period. The same can be visualized by selecting a year of the sales data, this show us that this data have a annual period too.

In [43]:
item_train_set['auto_corr'] = statsmodels.tsa.stattools.acf(item_train_set['Passengers In'].values, nlags = len(item_train_set), fft=False)

fig = px.line(item_train_set, x='date', y='auto_corr')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="todate"),
            dict(count=6, label="6m", step="month", stepmode="todate"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="todate"),
            dict(step="all")
        ])
    )
)
fig.show()


In [12]:
item_train_set

,date,store,item,sales,year,month,day,weekday,weekend,auto_corr
0,2013-01-01,1,1,13,2013,1,1,1,False,1.000000
1,2013-01-02,1,1,11,2013,1,2,2,False,0.446671
2,2013-01-03,1,1,14,2013,1,3,3,False,0.364925
3,2013-01-04,1,1,13,2013,1,4,4,False,0.344622
4,2013-01-05,1,1,10,2013,1,5,5,True,0.345692
...,...,...,...,...,...,...,...,...,...,...
1455,2016-12-26,1,1,16,2016,12,361,0,False,0.001057
1456,2016-12-27,1,1,10,2016,12,362,1,False,0.000815
1457,2016-12-28,1,1,16,2016,12,363,2,False,-0.000256
1458,2016-12-29,1,1,21,2016,12,364,3,False,-0.000772


## Stationary test
One of the most important analyses of time series data is to detect if the subject series is stationary or not, if not further transformations are required to remove trend and seasonality of the series, this analysis is even more important using statistic based models. Many authors of papers that use neural networks (NN) for time series defend that using this kind of machine learning model does not necessarily require the data to be already stationary. Although, is important to know if the data is stationary even if the model selected is a NN, because if the model does not converges well this could be a cause and know the properties of the series is highly valuable as well.

[1] Define a weak stationary time series if the mean function $ E[x(t)] $ is independent of $ t $, if the autocoraviation function $Cov (x(t+h), x(t))$ is independent of $ t $ for each $h$ and if $E[x^2[n]]$ is finite for each $n$.

To perform the weak stationary test, the mean function and the autocovariation function ware applied over rolling windows, since its sampled data. Thus, the window size has an impact over the functions interpretations, the window represents the interval in which the stationary hypothesis is tested.

Besides this definition, the $ statsmodels $ library has the Augmented Dickey-Fuller unit root test. The Augmented Dickey-Fuller test can be used to test for a unit root in a univariate process in the presence of serial correlation.

References
[1] Brockwell, Peter J., and Richard A. Davis. Introduction to time series and forecasting. springer, 2016.

In [13]:
def stationary_test(entry,delta=200,ad=False,std=False):
    window_size=int(len(entry)/15)
    # Weak stationary test
    # Mean function
    mean_y = []
    mean_x = []
    
    s_moment_y = []
    std_y = []
    n_data = len(entry)
    for i in range(0, int(n_data - window_size)):
        # Roling window start and end
        n_start = i
        n_end = n_start + window_size
        # Mean, standard deviation and second moment calculation
        mean_y_i = np.mean(entry[n_start:n_end])
        s_moment_y_i = moment(entry[n_start:n_end],moment=2)
        std_y_i = np.std(entry[n_start:n_end])
        # Saving the results 
        mean_y.append(mean_y_i)
        mean_x.append(n_end)
        s_moment_y.append(s_moment_y_i)
        std_y.append(std_y_i)

    # Autocovariance function
    acov_y = []
    acov_x = []
    n_data = len(entry)
    for i in range(0, int(n_data - window_size - delta)):
        n_start = i
        n_end = n_start + window_size
        acov_y_i = np.cov(
            entry[n_start:n_end], entry[n_start+delta:n_end+delta]
        )[0][0]
        acov_y.append(acov_y_i)
        acov_x.append(n_end)
    if(ad):
        result = adfuller(entry)
        print("ADF Statistic: %f" % result[0])
        print("p-value: {0}".format(result[1]))
        print("Critical Values:")
        for key, value in result[4].items():
            print("\t%s: %.3f" % (key, value))
        # if the p-value < 0.05  and the adf statistic is less than
        # critical values the series is stationary or is time independent
        
    return [mean_x,mean_y],[acov_x,acov_y], s_moment_y, std_y

In [14]:
# Weak stationary test
sales_train = item_train_set['sales'].to_numpy()

mean, cov, s_moment, std = stationary_test(sales_train, delta=20,ad=True) 
mean_value = np.zeros(len(sales_train))
mean_value[mean[0][0]:] = mean[1]

cov_value = np.zeros(len(sales_train))
cov_value[cov[0][0]:len(cov[1])+cov[0][0]] = cov[1]

std_value = np.zeros(len(sales_train))
std_value[mean[0][0]:] = std

s_moment_value = np.zeros(len(sales_train))
s_moment_value[mean[0][0]:] = s_moment

item_train_set['mean'] = mean_value
item_train_set['cov'] = cov_value
item_train_set['s_moment'] = std_value

fig = px.line(item_train_set, x='date', y=['mean', 'cov', 'sales', 's_moment'])
fig.show()

ADF Statistic: -2.819719
p-value: 0.05551174940647723
Critical Values:
	1%: -3.435
	5%: -2.864
	10%: -2.568


<ipython-input-14-a1d872bacd3c>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-14-a1d872bacd3c>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-14-a1d872bacd3c>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Analyzing the test results
The mean function test hypothesis failed, this can be noted by checking the function follows the trend and seasonlity of the sales series, the covariation function presents a linear behavior with a lot of disturbance, the standard deviation function seans to be linear like, but with a lot of noise. The second-moment function doesn't show a tendency to infinite on any $n$, that way passing on this condition.

Analyzing the functions plots it's correct to infer that this series of sales is non-stationary, this can be also noted on the autocorrelation plot, which shows a slow decaying behavior.the autocorrelation plot shows also a periodic behavior, which represents a periodic behavior on the time series, and also in several lags the ACF value goes above and bellow the confidence intervals of 99% and 95%

Besides the week sationary test, theAugmented Dickey-Fuller test can be done to check if the series of sales is stationary or not. This test checks if the series have a unit root and doing so it can make the assumption of how much the series is defined by it's trend.

There are 2 Hypothesis:

Null Hypothesis (H0): If failed to be rejected, it suggests the time series has a unit root, meaning it is non-stationary. It has some time dependent structure.

Alternate Hypothesis (H1): The null hypothesis is rejected; it suggests the time series does not have a unit root, meaning it is stationary. It does not have time-dependent structure.

We interpret this result using the p-value from the test. A p-value below a threshold (such as 5% or 1%) suggests we reject the null hypothesis (stationary), otherwise a p-value above the threshold suggests we fail to reject the null hypothesis (non-stationary).

p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

Since the p-value of the test value applied to the series is bigger than 0.05, the null hypothesis is rejected, thus the series is non-stationary


## Arima model

In [34]:
model = pm.ARIMA(order=(5, 1, 10), out_of_sample_size=pred_size)
model.fit(sales_train)
pred = np.array(model.predict(n_periods=pred_size))

/home/rodrigo/.local/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/home/rodrigo/.local/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [35]:
plot_df ={'Date': dates,
          'pred': pred,
          'label': sales_test,
          'error': pred-sales_test}

fig = px.line(plot_df, x='Date', y=['pred','label','error'])
fig.show()

In [36]:
mean_squared_error(sales_test, pred)

27.54152467840545